In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")


✅ Libraries imported successfully


In [2]:
np.random.seed(42)
n_samples = 5000

print("🔧 Generating realistic weather and flight data...")

# Common airport codes (mix of major hubs)
airports = ['JFK', 'LAX', 'ORD', 'DFW', 'ATL', 'DEN', 'SFO', 'SEA', 'MIA', 'BOS']

data = {
    # Airport information
    "departure_airport_code": np.random.choice(airports, n_samples),
    "arrival_airport_code": np.random.choice(airports, n_samples),
    
    # Temporal features
    "scheduled_departure_hour": np.random.randint(0, 24, n_samples),
    "month": np.random.randint(1, 13, n_samples),
    "season": np.random.choice(["Winter", "Spring", "Summer", "Fall"], n_samples),
    
    # Weather conditions (correlated features)
    "temperature": np.random.randint(-10, 40, n_samples),  # Celsius
    "wind_speed": np.random.randint(0, 80, n_samples),  # km/h
    "visibility": np.random.randint(100, 10000, n_samples),  # meters
    "precipitation_probability": np.random.randint(0, 101, n_samples),  # 0-100%
    "pressure": np.random.randint(980, 1040, n_samples),  # hPa
    "humidity": np.random.randint(20, 100, n_samples),  # percentage
    
    # Historical data
    "historical_delay_same_route": np.random.randint(0, 45, n_samples),  # avg minutes
}

df = pd.DataFrame(data)

# Add weather_condition based on other weather features (realistic correlation)
def assign_weather_condition(row):
    if row['precipitation_probability'] > 70 and row['temperature'] < 0:
        return 'Snow'
    elif row['precipitation_probability'] > 70:
        return 'Rain'
    elif row['wind_speed'] > 60:
        return 'Storm'
    elif row['visibility'] < 1000:
        return 'Fog'
    elif row['precipitation_probability'] > 40:
        return 'Cloudy'
    else:
        return 'Clear'

df['weather_condition'] = df.apply(assign_weather_condition, axis=1)

# Add time_of_day based on hour
def assign_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['time_of_day'] = df['scheduled_departure_hour'].apply(assign_time_of_day)

print(f"✅ Generated {len(df)} flight records")
print(f"\nWeather Conditions Distribution:")
print(df['weather_condition'].value_counts())
print(f"\nTime of Day Distribution:")
print(df['time_of_day'].value_counts())

df.head()



🔧 Generating realistic weather and flight data...
✅ Generated 5000 flight records

Weather Conditions Distribution:
Clear     1399
Rain      1180
Cloudy    1058
Storm      834
Snow       298
Fog        231
Name: weather_condition, dtype: int64

Time of Day Distribution:
Night        1588
Morning      1427
Afternoon    1108
Evening       877
Name: time_of_day, dtype: int64


,departure_airport_code,arrival_airport_code,scheduled_departure_hour,month,season,temperature,wind_speed,visibility,precipitation_probability,pressure,humidity,historical_delay_same_route,weather_condition,time_of_day
0,SFO,DFW,18,1,Winter,19,71,6195,42,996,67,38,Storm,Evening
1,DFW,ORD,20,2,Summer,19,72,3503,71,1012,33,2,Rain,Evening
2,SEA,BOS,2,4,Spring,14,54,6253,52,1035,38,9,Cloudy,Night
3,ATL,ATL,10,4,Spring,13,5,5650,4,1013,79,36,Clear,Morning
4,SFO,ATL,6,2,Spring,-1,69,3982,1,1036,41,18,Storm,Morning


In [3]:
print("\n🎯 Calculating flight delays based on weather factors...")

# Normalize features for delay calculation
wind_impact = (df['wind_speed'] / 80) * 100  # Strong winds = delays
visibility_impact = (1 - df['visibility'] / 10000) * 100  # Low visibility = delays
precip_impact = (df['precipitation_probability'] / 100) * 100
temp_extreme_impact = np.abs(df['temperature'] - 15) * 2  # Extreme temps = issues

# Weather condition severity scores
weather_severity = df['weather_condition'].map({
    'Clear': 0,
    'Cloudy': 10,
    'Rain': 30,
    'Fog': 40,
    'Snow': 60,
    'Storm': 80
})

# Time of day impact (morning and evening = more traffic)
time_impact = df['time_of_day'].map({
    'Morning': 15,
    'Afternoon': 5,
    'Evening': 20,
    'Night': 0
})

# Historical route delays
historical_impact = df['historical_delay_same_route'] * 0.8

# Seasonal impact
season_impact = df['season'].map({
    'Winter': 15,
    'Spring': 5,
    'Summer': 10,  # Thunderstorms
    'Fall': 8
})

# Calculate base delay score
delay_score = (
    0.25 * wind_impact +
    0.20 * visibility_impact +
    0.20 * precip_impact +
    0.10 * temp_extreme_impact +
    0.15 * weather_severity +
    0.05 * time_impact +
    0.03 * historical_impact +
    0.02 * season_impact
)

# Add realistic randomness
delay_score += np.random.normal(0, 10, n_samples)
delay_score = np.clip(delay_score, 0, 200)

# Convert to minutes with some non-linear scaling
df['delay_minutes'] = np.round(delay_score * (1 + np.random.uniform(-0.2, 0.2, n_samples)))
df['delay_minutes'] = np.clip(df['delay_minutes'], 0, 180)  # Cap at 180 minutes

# Create delay categories
def categorize_delay(minutes):
    if minutes == 0:
        return 'No Delay'
    elif minutes < 30:
        return 'Minor'
    elif minutes < 60:
        return 'Moderate'
    else:
        return 'Severe'

df['delay_category'] = df['delay_minutes'].apply(categorize_delay)

print("\n📊 Delay Statistics:")
print(df['delay_minutes'].describe())
print(f"\n📊 Delay Category Distribution:")
print(df['delay_category'].value_counts())

df[['weather_condition', 'wind_speed', 'visibility', 'delay_minutes', 'delay_category']].head(10)


# ============================================================================
# CELL 4: Prepare Features and Build Pipeline
# ============================================================================
print("\n🔧 Building ML pipeline...")

# Features and target
X = df.drop(columns=['delay_minutes', 'delay_category'])
y = df['delay_minutes']

# Define feature types
categorical_cols = [
    'departure_airport_code', 
    'arrival_airport_code', 
    'weather_condition', 
    'season', 
    'time_of_day'
]

numerical_cols = [
    'scheduled_departure_hour',
    'month',
    'temperature',
    'wind_speed',
    'visibility',
    'precipitation_probability',
    'pressure',
    'humidity',
    'historical_delay_same_route'
]

print(f"\nCategorical features ({len(categorical_cols)}): {categorical_cols}")
print(f"Numerical features ({len(numerical_cols)}): {numerical_cols}")

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ]
)

# Create ensemble model (combining Random Forest + Gradient Boosting)
from sklearn.ensemble import VotingRegressor

rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1
)

gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=8,
    learning_rate=0.1,
    random_state=42
)

# Ensemble voting regressor
ensemble_model = VotingRegressor(
    estimators=[
        ('rf', rf_model),
        ('gb', gb_model)
    ],
    weights=[0.6, 0.4]  # RF has slightly more weight
)

# Create pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', ensemble_model)
])

print("✅ Pipeline created successfully")


🎯 Calculating flight delays based on weather factors...

📊 Delay Statistics:
count    5000.000000
mean       40.389200
std        17.599729
min         0.000000
25%        28.000000
50%        40.000000
75%        52.000000
max       107.000000
Name: delay_minutes, dtype: float64

📊 Delay Category Distribution:
Moderate    2892
Minor       1359
Severe       707
No Delay      42
Name: delay_category, dtype: int64

🔧 Building ML pipeline...

Categorical features (5): ['departure_airport_code', 'arrival_airport_code', 'weather_condition', 'season', 'time_of_day']
Numerical features (9): ['scheduled_departure_hour', 'month', 'temperature', 'wind_speed', 'visibility', 'precipitation_probability', 'pressure', 'humidity', 'historical_delay_same_route']
✅ Pipeline created successfully


In [4]:
print("\n🚀 Training model...")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# Train
pipeline.fit(X_train, y_train)
print("✅ Training complete!")

# Predictions
y_pred = pipeline.predict(X_test)
y_pred = np.clip(y_pred, 0, 180)  # Ensure predictions are in valid range

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n" + "="*60)
print("📈 MODEL PERFORMANCE")
print("="*60)
print(f"\nMean Absolute Error (MAE): {mae:.2f} minutes")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} minutes")
print(f"R² Score: {r2:.4f}")

# Prediction distribution
print("\n📊 Prediction Distribution on Test Set:")
pred_bins = [0, 15, 30, 60, 120, 180]
pred_labels = ['0-15min', '15-30min', '30-60min', '60-120min', '120-180min']
pred_dist = pd.cut(y_pred, bins=pred_bins, labels=pred_labels)
print(pred_dist.value_counts().sort_index())

# Sample predictions vs actual
comparison = pd.DataFrame({
    'Actual': y_test[:10].values,
    'Predicted': np.round(y_pred[:10], 1)
})
comparison['Difference'] = np.abs(comparison['Actual'] - comparison['Predicted'])
print("\n📋 Sample Predictions (first 10):")
print(comparison)


# ============================================================================
# CELL 6: Test with Realistic Scenarios
# ============================================================================
print("\n" + "="*60)
print("🧪 TESTING WITH REALISTIC SCENARIOS")
print("="*60)

test_scenarios = [
    {
        "name": "✅ Perfect Conditions - Clear Summer Day",
        "data": {
            'departure_airport_code': 'JFK',
            'arrival_airport_code': 'LAX',
            'scheduled_departure_hour': 10,
            'month': 7,
            'temperature': 25,
            'wind_speed': 15,
            'visibility': 9000,
            'precipitation_probability': 5,
            'weather_condition': 'Clear',
            'pressure': 1013,
            'humidity': 45,
            'historical_delay_same_route': 5,
            'season': 'Summer',
            'time_of_day': 'Morning'
        }
    },
    {
        "name": "⚠️ Moderate Risk - Rainy Conditions",
        "data": {
            'departure_airport_code': 'ORD',
            'arrival_airport_code': 'ATL',
            'scheduled_departure_hour': 17,
            'month': 4,
            'temperature': 12,
            'wind_speed': 35,
            'visibility': 3000,
            'precipitation_probability': 75,
            'weather_condition': 'Rain',
            'pressure': 1005,
            'humidity': 85,
            'historical_delay_same_route': 20,
            'season': 'Spring',
            'time_of_day': 'Evening'
        }
    },
    {
        "name": "🔴 High Risk - Winter Storm",
        "data": {
            'departure_airport_code': 'DEN',
            'arrival_airport_code': 'BOS',
            'scheduled_departure_hour': 7,
            'month': 1,
            'temperature': -5,
            'wind_speed': 65,
            'visibility': 500,
            'precipitation_probability': 95,
            'weather_condition': 'Snow',
            'pressure': 995,
            'humidity': 90,
            'historical_delay_same_route': 35,
            'season': 'Winter',
            'time_of_day': 'Morning'
        }
    },
    {
        "name": "🌫️ Fog - Low Visibility",
        "data": {
            'departure_airport_code': 'SFO',
            'arrival_airport_code': 'SEA',
            'scheduled_departure_hour': 6,
            'month': 11,
            'temperature': 8,
            'wind_speed': 10,
            'visibility': 300,
            'precipitation_probability': 40,
            'weather_condition': 'Fog',
            'pressure': 1010,
            'humidity': 95,
            'historical_delay_same_route': 18,
            'season': 'Fall',
            'time_of_day': 'Morning'
        }
    },
    {
        "name": "⛈️ Storm - Severe Weather",
        "data": {
            'departure_airport_code': 'MIA',
            'arrival_airport_code': 'DFW',
            'scheduled_departure_hour': 15,
            'month': 8,
            'temperature': 32,
            'wind_speed': 75,
            'visibility': 2000,
            'precipitation_probability': 90,
            'weather_condition': 'Storm',
            'pressure': 990,
            'humidity': 88,
            'historical_delay_same_route': 28,
            'season': 'Summer',
            'time_of_day': 'Afternoon'
        }
    }
]

for scenario in test_scenarios:
    test_df = pd.DataFrame([scenario['data']])
    predicted_delay = pipeline.predict(test_df)[0]
    
    # Categorize
    if predicted_delay < 15:
        category = "No Delay"
        emoji = "✅"
    elif predicted_delay < 30:
        category = "Minor Delay"
        emoji = "🟡"
    elif predicted_delay < 60:
        category = "Moderate Delay"
        emoji = "🟠"
    else:
        category = "Severe Delay"
        emoji = "🔴"
    
    print(f"\n{scenario['name']}")
    print(f"  {emoji} Predicted Delay: {predicted_delay:.1f} minutes ({category})")


# ============================================================================
# CELL 7: Save the Model
# ============================================================================
model_filename = "weather_delay_model.pkl"
joblib.dump(pipeline, model_filename)

print("\n" + "="*60)
print("✅ MODEL SAVED SUCCESSFULLY!")
print("="*60)
print(f"\nFilename: {model_filename}")
print(f"Model Type: Ensemble Regressor (Random Forest + Gradient Boosting)")
print(f"Input Features: {len(categorical_cols) + len(numerical_cols)}")
print(f"Output: delay_minutes (0-180)")

print("\n📝 MODEL SPECIFICATIONS:")
print("="*60)
print("Model Name: weather_delay_model.pkl")
print("Algorithm: Ensemble (RF 60% + GB 40%)")
print("MAE: {:.2f} minutes".format(mae))
print("RMSE: {:.2f} minutes".format(rmse))
print("R² Score: {:.4f}".format(r2))

print("\n📋 REQUIRED FEATURES FOR PREDICTION:")
print("-" * 60)
print("Categorical:")
for col in categorical_cols:
    print(f"  - {col}")
print("\nNumerical:")
for col in numerical_cols:
    print(f"  - {col}")

print("\n✅ NEXT STEPS:")
print("="*60)
print("1. Download 'weather_delay_model.pkl'")
print("2. Place it in your Streamlit app directory")
print("3. Update your Streamlit app to use this model")
print("4. Test with different weather scenarios!")
print("\n🎉 Training Complete!")


🚀 Training model...
Training set: 4000 samples
Test set: 1000 samples
✅ Training complete!

📈 MODEL PERFORMANCE

Mean Absolute Error (MAE): 9.26 minutes
Root Mean Squared Error (RMSE): 11.48 minutes
R² Score: 0.5425

📊 Prediction Distribution on Test Set:
0-15min        14
15-30min      218
30-60min      689
60-120min      79
120-180min      0
dtype: int64

📋 Sample Predictions (first 10):
   Actual  Predicted  Difference
0    45.0       37.5         7.5
1    46.0       49.6         3.6
2    28.0       33.4         5.4
3    33.0       50.5        17.5
4    50.0       38.9        11.1
5    27.0       25.2         1.8
6    42.0       36.2         5.8
7    44.0       42.9         1.1
8    47.0       41.5         5.5
9    61.0       47.0        14.0

🧪 TESTING WITH REALISTIC SCENARIOS

✅ Perfect Conditions - Clear Summer Day
  ✅ Predicted Delay: 11.5 minutes (No Delay)

⚠️ Moderate Risk - Rainy Conditions
  🟠 Predicted Delay: 48.3 minutes (Moderate Delay)

🔴 High Risk - Winter Storm
  🔴 P